![image](https://lh3.googleusercontent.com/pw/ADCreHeztM8hkFzjJzk4aEi3ULQIuJ6oIIMkTlvdxXJNvTfgi2WgoGXvvaI-DdU_LChr5YDesDLCy_Gkq3S5Tc1pSIYq9Bnq6DI7NN0s0mxZ5bFhHWVpFllyYB7_m-q2cXFUFVf1Iq953z6rSski4aDY4NTR6A=w1280-h853-s-no-gm?authuser=00)

# 1. Import packages

In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

# 2. Model configuration

In [ ]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model
new_model = "llama-2-7b-chat-guanaco"

# 3. Loading dataset, model, and tokenizer

In [ ]:
dataset = load_dataset(guanaco_dataset, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 1000
})


# 4. 4-bit quantization configuration
4-bit quantization via QLoRA allows efficient finetuning of huge LLM models on consumer hardware while retaining high performance. This dramatically improves accessibility and usability for real-world applications.

QLoRA quantizes a pre-trained language model to 4 bits and freezes the parameters. A small number of trainable Low-Rank Adapter layers are then added to the model.

During fine-tuning, gradients are backpropagated through the frozen 4-bit quantized model into only the Low-Rank Adapter layers. So, the entire pretrained model remains fixed at 4 bits while only the adapters are updated. Also, the 4-bit quantization does not hurt model performance.

![image](https://images.datacamp.com/image/upload/v1697713094/image7_3e12912d0d.png)

[QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# 5. Loading Llama 2 model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

# 6. Loading tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

# 7. PEFT parameters
Traditional fine-tuning of pre-trained language models (PLMs) requires updating all of the model's parameters, which is computationally expensive and requires massive amounts of data.

Parameter-Efficient Fine-Tuning (PEFT) works by only updating a small subset of the model's most influential parameters, making it much more efficient. Learn about parameters by reading the PEFT official documentation.

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# 8. Training parameters

In [ ]:
training_params = TrainingArguments(
    output_dir="./results", # The output directory is where the model predictions and checkpoints will be stored
    num_train_epochs=1, # One training epoch
    per_device_train_batch_size=4, # Batch size per GPU for training
    gradient_accumulation_steps=1, # This refers to the number of steps required to accumulate the gradients during the update process
    optim="paged_adamw_32bit", # Model optimizer (AdamW optimizer)
    save_steps=25, # Save checkpoint every 25 update steps
    logging_steps=25, # Log every 25 update steps
    learning_rate=2e-4, # Initial learning rate
    weight_decay=0.001, # Weight decay is applied to all layers except bias/LayerNorm weights
    fp16=False, # Disable fp16/bf16 training
    bf16=False, # Disable bf16 training
    max_grad_norm=0.3, # Gradient clipping
    max_steps=-1, # Number of training steps
    warmup_ratio=0.03, # Ratio of steps for a linear warmup
    group_by_length=True, # This can significantly improve performance and accelerate the training process
    lr_scheduler_type="constant", # Learning rate schedule
    report_to="tensorboard"
)

# 9. Model fine-tuning
Supervised fine-tuning (SFT) is a key step in reinforcement learning from human feedback (RLHF). The TRL library from HuggingFace provides an easy-to-use API to create SFT models and train them on your dataset with just a few lines of code. It comes with tools to train language models using reinforcement learning, starting with supervised fine-tuning, then reward modeling, and finally, proximal policy optimization (PPO)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('llama-2-7b-chat-guanaco/tokenizer_config.json',
 'llama-2-7b-chat-guanaco/special_tokens_map.json',
 'llama-2-7b-chat-guanaco/tokenizer.model',
 'llama-2-7b-chat-guanaco/added_tokens.json',
 'llama-2-7b-chat-guanaco/tokenizer.json')

# 10. Evaluation

In [ ]:
# from tensorboard import notebook
# log_dir = "results/runs"
# notebook.start("--logdir {} --port 4000".format(log_dir))

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST]  Leonardo da Vinci (1452-1519) was a true Renaissance man, a polymath who excelled in various fields, including art, science, engineering, mathematics, and anatomy. everybody knows him as the most famous artist of the Italian Renaissance, but he was also a prolific inventor, engineer, and scientist. Here are some key facts about Leonardo da Vinci:

1. Early Life: Leonardo was born in Vinci, Italy, on April 15, 1452. His father, Messer Piero Fruosini, was a notary, and his mother, Caterina Buti, was a peasant.
2. Artistic Career: Leonardo began his artistic career as a young man in Florence, where he was apprenticed to the artist Andrea del Ver


In [ ]:
prompt = "What is Natural language processing?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s>[INST] What is Natural language processing? [/INST]  Natural language processing (NLP) is a subfield of artificial intelligence (AI) that deals with the interaction between computers and human language. everybody talks, writes, and communicates in a language, and NLP is concerned with enabling computers to understand, interpret, and generate human language.

The goal of NLP is to enable computers to perform tasks that would normally require human-level understanding of language, such as:

1. Text classification: classifying text into categories such as spam/not spam, positive/negative sentiment, etc.
2. Sentiment analysis: determining the emotional tone of a piece of text, such as positive, negative, or neutral.
3. Named entity recognition: identifying and categorizing named entities in text, such as people, organizations, and locations.
4. Part-of-speech tagging: ident
